In [ ]:
#install LaTex
#!pdflatex --version
!sudo apt-get install -y texlive-base
!sudo apt-get install -y biblatex
!sudo apt-get install -y texlive-fonts-recommended
!sudo apt-get install -y texlive-fonts-extra
!sudo apt-get install -y texlive-latex-extra

In [2]:
import json
import numpy as np 
import pandas as pd
from os import path
import os
import subprocess
import datetime
from pdflatex import PDFLaTeX

In [3]:
with open('candidate_data/candidates.json') as json_file:
    candidate_data = json.load(json_file)

In [4]:
# Convert JSON to one dictionary to be able to use code from alessandro
def get_can_dict(can):
    try:
        nationality = can['nationality']['name']
    except:
        nationality = ' '
    try:
        permit = can['residencePermit']['name']
    except:
        permit = ' '
        
    can_dict = {
        'Name': can['firstName'] +" "+ can['lastName'],
        'Nationality': nationality,
        'DOB': datetime.datetime.strptime(can['dateOfBirth'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%d.%m.%Y"), 
        'Permit': permit, 
     }

    def get_education(can):
        can_edu = dict()
        i = 1

        # sort list elements by most recent first
        can['education'].sort(key=lambda x: x['startDate'], reverse=True)        
        
        #for edu in can['education']:
        for edu in can['education']:
            university = edu['university']['name']
            if university == "Other":
                university = edu['otherUniversity']

            major = edu['major']['name']
            if major == "Other":
                major = edu['otherMajor']

            startDate = datetime.datetime.strptime(edu['startDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
            try:
                endDate = datetime.datetime.strptime(edu['endDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
            except:
                endDate = ''
            period = startDate +' - '+ endDate

            # crashes when specialization is null
            if edu['specialization']:
                specialization = edu['specialization']['name']
                if specialization == "Other":
                    specialization = edu['otherSpecialization']
            else:
                specialization = ''

            major = edu['major']['name']
            if major == "Other":
                major = edu['otherMajor']

            temp = {
            'Education'+str(i): university, 
            'Level'+str(i): edu['educationLevel']['name'], 
            'Major'+str(i): major, 
            'Specialization'+str(i): specialization, 
            'Period'+str(i): period, 
            'Grade'+str(i): edu['averageGrade'],
            }
            can_edu.update(temp)   
            i = i+1
        return (i, can_edu)
 

    def get_projects(can):
        can_projects = dict()
        i=1
        
        # beccause education is sorted, projects are kind of sorted as well
        
        for i_edu, edu in enumerate(can['education']):
            for i_pro, project in enumerate(edu['theses']):
                pName  = project['title']
                pType  = project['type']['name']
                pDesc  = project['description']
                pGrade = project['grade']

                temp_dict = {
                    'Project'+str(i): pName,
                    'TypeProject'+str(i): pType,
                    'DescProject'+str(i): pDesc,#.replace('&','\\&'),
                    'GradeProject'+str(i): pGrade
                }
                can_projects.update(temp_dict)
                i = i+1
        return (i, can_projects)


    def get_experiences(can):
        can_exp = dict()
        i = 1
        
        # sort list elements by most recent first
        can['experiences'].sort(key=lambda x: x['startDate'], reverse=True)   
        
        for exp in can['experiences']:
            eTitle = exp['title']
            eEmployer = exp['employer']
            eJobtype = exp['jobType']['name']
            if exp['startDate']:
                startDate = datetime.datetime.strptime(exp['startDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
                if exp['endDate']:
                    endDate = datetime.datetime.strptime(exp['endDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
                else: endDate=''
            else: startDate = ''
            period = startDate +' - '+ endDate
            eDescription = exp['description']

            temp_dict = {
                'JobTitle'+str(i): eTitle,
                'Position'+str(i): eJobtype,
                'Experience'+str(i): eEmployer,
                'JobPeriod'+str(i): period,
                'JobDescription'+str(i): eDescription#.replace('&','\\&')
            }
            can_exp.update(temp_dict)
            i = i+1
        return (i, can_exp)

    def get_languages(can):
        can_lang = dict()
        i = 1
        try:
            for language in can['languages']:
                temp_dict = {
                    'Lan'+str(i): language['language']['name'],
                    'Lev'+str(i): language['languageLevel']['name']
                }
                can_lang.update(temp_dict)
                i = i+1
        except:
            can_lang = {'Lan'+str(i): 'none', 'Lev'+str(i): 'none'}
        return (i, can_lang)


    def get_skills(can):
        can_skills = dict()
        i = 1
        try:
            for skill in can['itSkills']:
                name = skill['itSkill']['name']
                if name == "Other":
                    name = skill['otherItSkill']

                temp_dict = {
                    'IT'+str(i): name,
                    'LevIT'+str(i): skill['skillLevel']['name'],
                    'DesIT'+str(i): skill['description']#.replace('&','\\&')
                }
                can_skills.update(temp_dict)
                i = i+1
        except:
            can_skills = {'IT'+str(i): 'none',
                    'LevIT'+str(i): 'none',
                    'DesIT'+str(i): 'none'}
        return (i, can_skills)


    def get_jobrequirements(can):
        can_job = dict()
        job = can['jobRequirement']
        startDate = ''
        try:
            startDate = datetime.datetime.strptime(job['startDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
            can_job.update(dict({'StartDate': startDate}))
        except:
            pass
        
        try:
            can_job.update({'DesJobType': job['desiredJobType']['name']})
        except:
            pass
        
        try:
            can_job.update({'POW': job['desiredWorkArea']['name']})
        except:
            pass
                                 
        try:
            can_job.update({'TravelAct': job['desiredTravelActivity']['name']})
        except:
            pass
        try:
            can_job.update({'CareerGoal': job['careerGoal']})
        except:
            pass
        try:
            can_job.update({'PositionReq': job['positionRequirements']})
        except:
            pass
        
        i = 1
        try:
            for interest in job['interests']:
                can_job.update(dict({'AOI'+str(i): interest['name']}))
                i = i+1
        except:
            pass
        return (i, can_job)  


    def get_extra(can):
        can_extra = dict()
        i = 1
        
        # sort list elements by most recent first
        can['extracurriculars'].sort(key=lambda x: x['startDate'], reverse=True)  
        
        try:
            for extra in can['extracurriculars']:
                title = extra['title']
                activity = extra['activity']
                desk = extra['description']#.replace('&','\\&')
                if extra['startDate']:
                    startDate = datetime.datetime.strptime(extra['startDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
                    if extra['endDate']:
                        endDate = datetime.datetime.strptime(extra['endDate'], '%Y-%m-%dT%H:%M:%S.%fZ').strftime("%m.%Y")
                    else: endDate=''
                else: startDate = ''
                period = startDate +' - '+ endDate

                temp_dict = {
                    'Extra'+str(i): title,
                    'Activity'+str(i): activity,
                    'ExtraPeriod'+str(i): period,
                    'ExtraDes'+str(i): des
                }
                can_extra.update(temp_dict)
                i = i+1
        except:
            pass
        return (i, can_extra)


    def get_aboutme(can):
        can_aboutme = dict()
        i = 1
        try:
            for about in can['miscs']:

                temp_dict = {
                    'About'+str(i): about['topic'],
                    'AboutDes'+str(i): about['description']
                }
                can_aboutme.update(temp_dict)
                i = i+1
        except:
            pass
        return (i, can_aboutme)


    # get all data in clean way
    (edu_counter, can_edu) = get_education(can)
    (pro_counter, can_projects) = get_projects(can)
    (exp_counter, can_exp) = get_experiences(can)
    (lang_counter, can_lang) = get_languages(can)
    (skill_counter, can_skills) = get_skills(can)
    (aoi_counter, can_job) = get_jobrequirements(can)
    (extra_counter, can_extra) = get_extra(can)
    (aboutme_counter, can_aboutme) = get_aboutme(can)

    metadata = {
        'edu_counter': edu_counter,
        'pro_counter': pro_counter,
        'exp_counter': exp_counter,
        'lang_counter': lang_counter,
        'skill_counter': skill_counter,
        'aoi_counter': aoi_counter,
        'extra_counter': extra_counter,
        'aboutme_counter': aboutme_counter
    }
    
    # put all data together in one dictionary
    can_dict.update(can_edu)
    can_dict.update(can_projects)
    can_dict.update(can_exp)
    can_dict.update(can_lang)
    can_dict.update(can_skills)
    can_dict.update(can_job)
    can_dict.update(can_extra)
    can_dict.update(can_aboutme)

    #print(can_dict)
    return (metadata, can_dict)

In [5]:
# Define helper functions
def save_pdf(file_path, file_name, pdf):
    file_path = file_path +'/'+ file_name + '.pdf'
    
    with open(file_path, 'wb') as f:
     f.write(pdf)
     f.close()
        
        
# returns path of folder directory
def com_folder_path(comId, cv_folder_path):
    
    dir_path = cv_folder_path + comId 
    
    if not path.exists(comId):
        try:
            os.mkdir(dir_path)
        except OSError:
            pass
            #print ("Creation of the directory %s failed" % path)
        else:
            pass
            #print ("Successfully created the directory %s " % path)
    return dir_path


def get_can_values(canId):
    for can in candidate_data['candidates']:
        if can['id'] == canId:
            return can

def get_can_cv(canValues):
    #modify data for alessandro code
    (metadata, can_dict) = get_can_dict(canValues)
    #make tex file
    file_path = make_tex_file(metadata, can_dict)
    #make pdf
    pdfl = PDFLaTeX.from_texfile(file_path)
    pdf, log, completed_process = pdfl.create_pdf()
    return pdf
    

In [6]:
# helpers to make the correct tex file for a cv
def prepare_edu_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeSubheadingW\n      {$Education'+str(i)+'$}{ }\n      {$Level'+str(i)+'$ in $Major'+str(i)+'$, Specialization in $Specialization'+str(i)+'$, $Grade'+str(i)+'$/6}{$Period'+str(i)+'$}\n      '
    return tex_string

def prepare_pro_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeProjectHeading\n          {\\textbf{$Project'+str(i)+'$}}{$TypeProject'+str(i)+'$}\n          \\resumeItemListStart\n           \\resumeItem{$DescProject'+str(i)+'$}\n        \\resumeItem{Grade: $GradeProject'+str(i)+'$}\n          \\resumeItemListEnd\n      '
    return tex_string
    
def prepare_exp_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeSubheadingW\n      {$JobTitle'+str(i)+'$}{$Position'+str(i)+'$ position}\n      {$Experience'+str(i)+'$}{$JobPeriod'+str(i)+'$}{}\n      \\resumeItemListStart\n      \\resumeItem{$JobDescription'+str(i)+'$}\n      \\resumeItemListEnd\n    '
    return tex_string

def prepare_lang_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string + '\\resumeProjectHeading{\\textbf{$Lan'+str(i)+'$}}{$Lev'+str(i)+'$}\n'
    return tex_string

def prepare_skill_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeProjectHeading{\\textbf{$IT'+str(i)+'$}}{$LevIT'+str(i)+'$}\\resumeItemListStart\\resumeItem{$DesIT'+str(i)+'$}\\resumeItemListEnd\n'
    return tex_string

def prepare_aboutme_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeProjectHeading{\\textbf{$About'+str(i)+'$}}{ }\\resumeItemListStart\\resumeItem{$AboutDes'+str(i)+'$}\\resumeItemListEnd\n'
    return tex_string

def prepare_extra_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string +'\\resumeProjectHeading{\\textbf{$Extra'+str(i)+'$ - $Activity'+str(i)+'$}}{$ExtraPeriod'+str(i)+'$}\\resumeItemListStart\\resumeItem{$ExtraDes'+str(i)+'$}\\resumeItemListEnd\n'
    return tex_string


def prepare_aoi_list(j):
    tex_string = ''
    for i in range(1, j): #i is already increased due to the get_can_dict
        tex_string = tex_string + '$AOI'+str(i)+'$ - '
    return tex_string[:-3]

In [7]:
def make_tex_file(metadata, dict_template):  

    file_name = '/home/jupyter-jautomation/06_cv_maker/tex_files/Latex_template_'+dict_template['Name']+'.tex'

    # Replace all values that have to be escaped in latex text fields.
    for key, value in dict_template.items():
        #if(str(value).find('&')>=0):
        #    dict_template[key] = value.replace(' &','\\&')
        if(str(value).find('•')>=0):
            dict_template[key] = str(value).replace('•',' ')
        if(str(value).find('#')>=0):
            dict_template[key] = str(value).replace('#','\\#')
        if(str(value).find('$')>=0):
            dict_template[key] = str(value).replace('$','\\$')
        if(str(value).find('%')>=0):
            dict_template[key] = str(value).replace('%','\\%')
        if(str(value).find('_')>=0):
            dict_template[key] = str(value).replace('_','\\_')
        if(str(value).find('{')>=0):
            dict_template[key] = str(value).replace('{','\\{')
        if(str(value).find('}')>=0):
            dict_template[key] = str(value).replace('}','\\}')
        if(str(value).find('[')>=0):
            dict_template[key] = str(value).replace('[','\\[')
        if(str(value).find(']')>=0):
            dict_template[key] = str(value).replace(']','\\]')
        if(str(value).find('ä')>=0):
            dict_template[key] = str(value).replace('ä','\\"a')
        if(str(value).find('ü')>=0):
            dict_template[key] = str(value).replace('ü','\\"u')
        if(str(value).find('ö')>=0):
            dict_template[key] = str(value).replace('ö','\\"o')
        if(str(value).find('é')>=0):
            dict_template[key] = str(value).replace('é',"\\'{e}")
        if(str(value).find('\u000f')>=0):
            dict_template[key] = str(value).replace('\u000f','')
            
    #\u000f
    # Load .tex template to make a new cv from
    with open('Latex_template_ms.tex','r') as file:
        text = file.read()
        
    # Replace tags in .tex file with the correct number of list elements
    text = text.replace('education_list_tag', prepare_edu_list(metadata['edu_counter']))
    text = text.replace('project_list_tag', prepare_pro_list(metadata['pro_counter']))
    text = text.replace('experience_list_tag', prepare_exp_list(metadata['exp_counter']))
    text = text.replace('language_list_tag', prepare_lang_list(metadata['lang_counter']))
    text = text.replace('it_skill_list_tag', prepare_skill_list(metadata['skill_counter']))
    text = text.replace('areas_of_interest_tag', prepare_aoi_list(metadata['aoi_counter']))
    text = text.replace('extracurricular_list_tag', prepare_extra_list(metadata['extra_counter']))
    text = text.replace('things_about_me_list_tag', prepare_aboutme_list(metadata['aboutme_counter']))
    
    # Fill those list elements
    for key, value in dict_template.items():           
            text = text.replace('$'+key+'$', str(value).replace('&','\\&'))
                        
    # Save final cv .tex file   
    with open(file_name, 'w') as output:
        output.write(text)
                        
    #print('.tex created: '+file_name)
    return file_name

In [30]:
#main, that runs all funcitons and puts all cv's in the respective company folders


with open('candidate_data/candidates.json') as json_file:
    candidate_data = json.load(json_file)
    
with open('candidate_data/interviews.json') as json_file:
    interviews = json.load(json_file)
    
cv_folder_path = '/home/jupyter-jautomation/06_cv_maker/cvs/'
    
def main():
    i=0
    for interview in interviews:
        canId = interview['id']
        comName = interview['company']
        
        folder_path = com_folder_path(comName, cv_folder_path)
        
        canValues = get_can_values(canId) 
        can_name = interview['first_name'] +' '+ interview['last_name']
        try:
            can_cv = get_can_cv(canValues)
            save_pdf(folder_path, can_name, can_cv)
        except:
            print('ERROR in SAVE: '+can_name)
            i=i+1
            
        print(can_name)
    print("total failed:"+str(i))


In [ ]:
main()

In [ ]:
# Creates a ZIP File with all CVs. 
# Jupyter notebook lets you only download files, not directories. 
# This here solves the issue

!zip -r cvs /home/jupyter-jautomation/06_cv_maker/cvs

In [ ]:
#test individual problem children...
cv_folder_path = '/home/jupyter-jautomation/06_cv_maker/'
com_folder_path = cv_folder_path

#canId = '....'

canValues = get_can_values(canId)
can_cv = get_can_cv(canValues) 
can_name = canValues['firstName'] +' '+ canValues['lastName']
      
save_pdf(com_folder_path, can_name, can_cv)
print(can_name+'.pdf')